In [1]:
import os
import sys
import pandas as pd
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from code_utils.read_ipcc_bib import read_bib_wg
from code_utils.clean_doi import get_doi_cleaned
from code_utils.enriching_data import get_countries

Read IPCC references in .bib files

In [ ]:
wgs={'1':{'wg1':[], 'dataframes_1':{}, 'listdir1':os.listdir(module_path+f"\\IPCC_bibliography\\AR6\\WG1")},
     '2':{'wg2':[], 'wg2_update':[], 'dataframes_2':{}, 'listdir2':os.listdir(module_path+f"\\IPCC_bibliography\\AR6\\WG2")},
     '2_CROSS':{'wg2_CROSS':[], 'wg2_CROSS_update':[], 'dataframes_2_CROSS':{}, 'listdir2_CROSS':os.listdir(module_path+f"\\IPCC_bibliography\\AR6\\WG2_CROSS")},
     '3':{'wg1':[], 'dataframes_3':{}, 'listdir3':os.listdir(module_path+f"\\IPCC_bibliography\\AR6\\WG3")}}

In [ ]:
for k in ['1','2','2_CROSS','3']:
    wgs[k][f'df_{k}'] = read_bib_wg(wgs,k,verbose=True)
    print(wgs[k][f'wg{k}_update'])
    #wgs[k][f'df_{k}'].to_json(module_path+f'\\IPCC_bibliography\\AR6\\structured_data\\data_wg{k}.jsonl', orient='records', lines=True)

Read and clean the data from 'structured_data' folder

In [2]:
data_all={}

In [3]:
for k in ['1','2','2_CROSS','3']:
   data_all[f'df_wg{k}']=pd.read_json(module_path+f'\\IPCC_bibliography\\AR6\\structured_data\\data_wg{k}.jsonl', lines= True)

In [4]:
#all data 
df_ipcc=pd.concat(list(data_all.values()), ignore_index=True)

In [5]:
df_ipcc['doi']=df_ipcc.loc[:,'doi'].apply(lambda x: get_doi_cleaned(x))

Enriching data through OpenAlex API integration

In [ ]:
df_unique_doi=df_ipcc.groupby(subset=['doi'])" faire groupby avec chap et frequence d'apparition"
df_unique_doi.loc[:,'doi']=df_unique_doi.loc[:,'doi'].apply(lambda x: None if str(x)[:4]=='http' else x)
df_unique_doi=df_unique_doi.dropna(subset=['doi'])

In [ ]:
#get data_OpenAlex from the API (6h)
""" 
df_unique_doi.apply(lambda row: get_open_alex_data(json_OA,row['doi']), axis= 1)
pd.DataFrame(json_OA).to_json(path+'data_OpenAlex.json', orient='records') 
"""

In [6]:
#get data_OpenAlex from the file
data_OpenAlex_all=pd.read_json(module_path+f'\\IPCC_bibliography\\AR6\\structured_data\\data_OpenAlex.json')

In [ ]:
data_OpenAlex_all[data_OpenAlex_all.doi==]

In [ ]:
countries_list = []
for i,row in data_OpenAlex_all.iterrows():
    countries=get_countries(data_OpenAlex_all,row)
    countries_list.append(countries)
data_OpenAlex_all['countries'] = countries_list